In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import json
import os

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.8,
)
mp_drawing = mp.solutions.drawing_utils

def create_landmark(output_folder="data/landmarks_raw"):

    os.makedirs(output_folder, exist_ok=True)

    label = input("Add gesture name")

    cap = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(frame_rgb)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Рисуем точки на руке
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Извлекаем координаты
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])

                # Сохраняем JSON
                data = {
                    "landmarks": landmarks,
                    "label": label
                }
                with open(f"data/landmarks_raw/{label}_{count}.json", "w") as f:
                    json.dump(data, f)
                count += 1

        cv2.imshow("Hand Capture", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
